In [66]:
#Importing Libraries

from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
import requests
import lxml.html as lh
import pandas as pd

In [67]:
#IDs data imported
path="C:/Users/Pandey/Desktop/Bhai/Revenue Controls and Migration Check.xlsx"
df=pd.read_excel(path)
list_of_acco_ids = list(df['Acco-Ids'].unique())
last_ten_acco_ids = list_of_acco_ids[-11:]

In [68]:
#LOGIN
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.oyo-vacation-homes.net/cgi/lars/algm/login.htm")
driver.find_element_by_xpath('//*[@id="loginid"]').send_keys('')
driver.find_element_by_xpath('//*[@id="passwd"]').send_keys('')
driver.find_element_by_xpath('//*[@id="signin"]').click()

In [72]:
#SEARCH
#key='IT-00055-00'
#key='NL-0036-87'
key='AT-9631-35'
driver.find_element_by_xpath('//*[@id="quicksearchmenu"]').send_keys(Keys.CONTROL, 'a')
driver.find_element_by_xpath('//*[@id="quicksearchmenu"]').send_keys(key)
driver.find_element_by_xpath('//*[@id="quicksearchmenu"]').send_keys(Keys.RETURN)

In [85]:
#Status Check
table =  driver.find_element_by_xpath("//*[@class='table table-light']") 

data = [item.text for item in table.find_elements_by_xpath(".//*[self::td or self::th]")]
#print(data)
del data[3::4]

abb_status = data[5]
ha_status = data[8]
bdc_status = data[11]
status={}
status[key]=[abb_status,ha_status,bdc_status]

In [86]:
status

{'AT-9631-35': ['ONLINE', 'ONLINE', 'NOT CONNECTED']}

In [90]:
#Website Appearances

driver.find_element_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "manageWebsites", " " ))]').click()
el = driver.find_element_by_class_name('icheck-list')
list_wa = []
list_ota = ['Eurorelais','Casamundo','Booking.com','HomeAway','Traum Ferienwohnungen','AirBnb','Others']
dict_wa = {}
to_append = None
for wa in el.find_elements_by_tag_name('div'):
    
    if(wa.get_attribute("class")=='icheckbox_minimal-blue checked'):
        to_append= 'BLOCKED'
    else:
        to_append= 'UNBLOCKED'
    
    list_wa.append(to_append)
        
    #print(wa.get_attribute("class"))

dict_wa = dict(zip(list_ota, list_wa))
    

In [91]:
dict_wa

{'Eurorelais': 'UNBLOCKED',
 'Casamundo': 'UNBLOCKED',
 'Booking.com': 'BLOCKED',
 'HomeAway': 'UNBLOCKED',
 'Traum Ferienwohnungen': 'UNBLOCKED',
 'AirBnb': 'UNBLOCKED',
 'Others': 'UNBLOCKED'}

In [93]:
driver.find_element_by_xpath('//*[(@id = "dowebsitecheck")]').click()


In [137]:
el = driver.find_element_by_id("dowebsitecheck_modal_body")
c=0
OTA_check= ""
for wa in el.find_elements_by_tag_name('div'):
    #print(wa.text)
    #print(c)
    c=c+1
    if(c==6 or c==1 or c==11):
        #print(wa.text)
        OTA_check= OTA_check + (wa.text)
OTA_check = OTA_check.replace('\n',' - ').split(' - VIEW DETAILS')[:3]
OTA_check_dict={}
abb_check = OTA_check[1]
ha_check = OTA_check[0]
bdc_check = OTA_check[2]
OTA_check_dict[key] = [abb_check,ha_check,bdc_check]

In [138]:
OTA_check_dict

{'AT-9631-35': ['AirBnb - No errors. Can be published on AirBnb.',
  'HomeAway / Expedia - No errors. Can be published on HomeAway / Expedia.',
  'Booking.com - Not to Booking.com option checked on Accommodation level']}